## Bot experto en leyes electorales

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# Importamos LangChain, Chroma, Plotly y herramientas necesarias

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
# Nombre del directorio donde se guardará la base de datos vectorial
db_name = "vector_db"

In [4]:
# Cargar las variables de entorno desde el archivo llamado .env
load_dotenv(override=True)  

# Obtener la clave de la API de OpenAI desde las variables de entorno
# Si no se encuentra, se asigna un valor por defecto (útil si no estás usando un archivo .env)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY',  'your-key-if-not-using-env')


In [ ]:
# Cargar documentos usando los cargadores (loaders) de LangChain
# Se toma todo lo que esté en las subcarpetas dentro de "knowledge-base"
folders = glob.glob("knowledge-base/*")

# Función para agregar un tipo de documento como metadato.
# Esto permite identificar más adelante de qué subcarpeta provino cada documento.
def agregar_metadatos(doc, tipo_doc):
    doc.metadata["doc_type"] = tipo_doc
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

# Lista donde se almacenarán todos los documentos cargados
documents = []

# Recorremos cada carpeta dentro de la base de conocimiento
for folder in folders:
    # El tipo de documento se extrae del nombre de la carpeta
    tipo_doc = os.path.basename(folder)
    
    # Se crea un cargador para todos los archivos Markdown (*.md) dentro de la carpeta
    loader = DirectoryLoader(
        folder,
        glob="**/*.md",
        loader_cls=TextLoader,
        loader_kwargs=text_loader_kwargs
    )
    
    # Se cargan los documentos desde esa carpeta
    folder_docs = loader.load()
    
    # Se agregan los metadatos y se agregan a la lista principal
    documents.extend([agregar_metadatos(doc, tipo_doc) for doc in folder_docs])

# Se define cómo se dividirán los documentos en fragmentos (chunks)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Versión alternativa de splitter, mejora las separaciones porque para dividir intenta respetar las jerarquías (títulos, subtítulos, párrafos, oraciones...):
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=300,
#     separators=["\n# ", "\n## "]
# )

chunks = text_splitter.split_documents(documents)

# Imprime la cantidad total de fragmentos generados
print(f"Número total de fragmentos: {len(chunks)}")

# Imprime los tipos de documentos (carpetas origen) que se encontraron
print(f"Tipos de documento encontrados: {set(doc.metadata['doc_type'] for doc in documents)}")


Created a chunk of size 1931, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1373, which is longer than the specified 1000
Created a chunk of size 2203, which is longer than the specified 1000
Created a chunk of size 1633, which is longer than the specified 1000
Created a chunk of size 1095, which is longer than the specified 1000
Created a chunk of size 1188, which is longer than the specified 1000
Created a chunk of size 1197, which is longer than the specified 1000
Created a chunk of size 1420, which is longer than the specified 1000
Created a chunk of size 2569, which is longer than the specified 1000
Created a chunk of size 2760, which is longer than the specified 1000
Created a chunk of size 1307, which is longer than the specified 1000
Created a chunk of size 1212, which is longer than the specified 1000


Created a chunk of size 1870, which is longer than the specified 1000
Created a chunk of size 1349, which is longer than the specified 1000
Created a chunk of size 2385, which is longer than the specified 1000
Created a chunk of size 2614, which is longer than the specified 1000
Created a chunk of size 1801, which is longer than the specified 1000
Created a chunk of size 1206, which is longer than the specified 1000
Created a chunk of size 1146, which is longer than the specified 1000
Created a chunk of size 2339, which is longer than the specified 1000
Created a chunk of size 1468, which is longer than the specified 1000
Created a chunk of size 1223, which is longer than the specified 1000
Created a chunk of size 1744, which is longer than the specified 1000
Created a chunk of size 2022, which is longer than the specified 1000
Created a chunk of size 1796, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of s

Número total de fragmentos: 263
Tipos de documento encontrados: {'Manual autoridades de mesa', 'Código electoral', 'Candidatos'}


## Armamos los embeddings

In [6]:
# Importa el modelo de embeddings de OpenAI (usa representaciones vectoriales del texto)
embeddings = OpenAIEmbeddings()

# Para usar embeddings gratuitos de HuggingFace (por ejemplo, el modelo "all-MiniLM-L6-v2"),
# reemplazar la línea anterior con estas dos:
#from langchain.embeddings import HuggingFaceEmbeddings
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Si ya existe una base de datos vectorial con ese nombre, se elimina.
# Esto se hace para evitar conflictos o duplicados al rehacer el proceso desde cero.
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Crea una nueva base de datos vectorial (vectorstore) a partir de los documentos fragmentados.
# Cada fragmento es convertido en un vector usando el modelo de embeddings.
# La base de datos se guarda en el directorio especificado por 'db_name'.
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=db_name
)

# Imprime cuántos documentos (fragmentos de texto) fueron almacenados en la base de datos vectorial.
print(f"Base de datos vectorial creada con {vectorstore._collection.count()} documentos")

Base de datos vectorial creada con 263 documentos


In [7]:
# Vamos a investigar los vectores almacenados

# Se accede directamente a la colección subyacente del almacén vectorial
collection = vectorstore._collection

# Se obtiene la cantidad total de vectores almacenados
count = collection.count()

# Se extrae un vector de ejemplo (el primero) incluyendo sus embeddings (representación numérica)
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Se calcula cuántas dimensiones tiene ese vector
dimensions = len(sample_embedding)

# Se imprime cuántos vectores hay y cuántas dimensiones tiene cada uno
print(f"Hay {count:,} vectores con {dimensions:,} dimensiones en la base de datos vectorial")


Hay 263 vectores con 1,536 dimensiones en la base de datos vectorial


## Visualización

In [8]:
# Obtener los vectores, textos y metadatos de la base
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']

# Extraer los tipos de documento desde los metadatos
doc_types = [metadata['doc_type'] for metadata in metadatas]

# Asignar un color por tipo de documento
color_map = {
    "Candidatos": "blue",
    "Código electoral": "green",
    "Manual autoridades de mesa": "red"
}
colors = [color_map.get(t, "gray") for t in doc_types]  # 'gray' por si aparece alguno no esperado


In [10]:
# Reducción de dimensionalidad a 2D con t-SNE
tsne_2d = TSNE(n_components=2, random_state=42)
reduced_vectors_2d = tsne_2d.fit_transform(vectors)

# Visualización 2D
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors_2d[:, 0],
    y=reduced_vectors_2d[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Tipo: {t}<br>Texto: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Visualización 2D de la base vectorial (Chroma)',
    xaxis_title='x',
    yaxis_title='y',
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [12]:
# Reducción de dimensionalidad a 3D
tsne_3d = TSNE(n_components=3, random_state=42)
reduced_vectors_3d = tsne_3d.fit_transform(vectors)

# Visualización 3D
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors_3d[:, 0],
    y=reduced_vectors_3d[:, 1],
    z=reduced_vectors_3d[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Tipo: {t}<br>Texto: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Visualización 3D de la base vectorial (Chroma)',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()



## Unimos todo con LangChain

In [13]:
# El precio es un factor importante para nuestra empresa, por lo tanto vamos modelos de bajo costo

# Crear un nuevo modelo de chat con OpenAI
#MODEL = "gpt-4o-mini"  # Este modelo es una versión más económica dentro de la familia GPT-4o
#llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Alternativa: Para usar Ollama localmente, descomentá esta línea en su lugar
llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# Configurar la memoria de conversación para el chat
# Esto permite que el modelo recuerde lo que se dijo anteriormente en la conversación
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# El "retriever" es una abstracción sobre el VectorStore que se usará durante RAG (Retrieval-Augmented Generation)
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# Integración de todos los componentes:
# Se crea una cadena de conversación que incluye:
# - el modelo de lenguaje (llm)
# - el sistema de recuperación de información (retriever)
# - la memoria conversacional (memory)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11224\3188444933.py:12: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [14]:
print("Total embeddings indexados:", vectorstore._collection.count())


Total embeddings indexados: 263


In [15]:
from collections import Counter
cnt = Counter(chunk.metadata.get("doc_type") for chunk in chunks)
print(cnt)


Counter({'Código electoral': 237, 'Candidatos': 20, 'Manual autoridades de mesa': 6})


In [16]:
# ¿están todos los doc_type en el vectorstore?
for doc_type, _ in cnt.items():
    example = next(ch for ch in chunks if ch.metadata["doc_type"] == doc_type)
    # intenta buscar ese chunk:
    res = vectorstore.similarity_search_with_score(example.page_content[:50], k=1)
    print(doc_type, res)


Candidatos [(Document(id='ee4f6012-6b79-4987-897e-bef453679d26', metadata={'doc_type': 'Candidatos', 'source': 'knowledge-base\\Candidatos\\Lista completa candidatos - Código 1.md'}, page_content='# Lista 1 – MOVIMIENTO DE INTEGRACIÓN Y DESARROLLO\n\n**Candidatos (lista completa)**\n\n1. Caruso Lombardi Ricardo Daniel  \n2. Villar Agustina  \n3. Arancio Miguel Angel  \n4. Lernoud María del Pilar  \n5. Testori Schroeder Pablo César Luciano  \n6. Radice Andrea Carla  \n7. Vega Osvaldo Rubén  \n8. Young Valeria Verónica  \n9. Devita Rodrigo Nahuel  \n10. De Santi Mónica Emilia  \n11. Peña Rodrigo  \n12. Rotoli María Eugenia  \n13. Mosquera Andrés Alberto  \n14. Arias Pereyra Gabriela Laura  \n15. Uz Juan Ignacio  \n16. Albrecht María Alejandra  \n17. Baletti Lucas Agustín  \n18. Herrera Diana Melisa  \n19. Buletti Carlos Alberto  \n20. Calabrese Sandra María  \n21. Mario Castiñeira Christian Diego  \n22. Nappa Sol Mariana  \n23. Ravanetti Américo Marcelo  \n24. Pace Madelaine Aixa  \n25. 

In [17]:
# Probamos una pregunta (no tiene el prompt de sistema)

query = "¿Puedo ir a votar disfrazado de un candidato?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

No, no es posible votar disfrazado de un candidato. En la mayoría de los países democráticos, el voto es un derecho ciudadano que se ejerce de manera directa y personal, y se requiere una identificación específica para garantizar la autenticidad del voto.

En las elecciones, los votantes deben presentarse con su credencial o documento de identidad correspondiente, y deben identificarse ante los funcionarios electorales. El disfrazarse de un candidato o intentar votar en nombre de otra persona es una forma de fraude electoral y está estrictamente prohibido por la ley.

Si estás interesado en apoyar a un candidato específico, lo mejor es hacerlo de manera abierta y respetuosa, participando en las discusiones políticas, difundiendo información sobre su programa y políticas, o voluntariéndote como voluntario en su campaña. De esta manera, puedes expresar tu opinión y apoyo de manera ética y responsable.

Es importante recordar que el voto es un derecho y una responsabilidad ciudadana, y de

In [18]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

# Añadimos esta clase para investigar qué se envía "detrás de escena" durante una consulta
from langchain_core.callbacks import StdOutCallbackHandler

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

#Prompt de sistema. 
# Obs: Podría mejorarse: Es algo antipático, si se le dice hola dice que no encuentra la respuesta. Pero no es por el prompt sino que siempre busca contexto
template = """You are **ElectoAI**, an expert electoral assistant for the **2025 legislative elections in the City of Buenos Aires (CABA)**.

**Role & Scope**  
- Proporcionás respuestas **claras**, **precisas** y **neutrales** sobre procedimientos electorales, normativas vigentes, situaciones prácticas en el lugar de votación y candidaturas, basándote únicamente en la información provista en el contexto recuperado.  
- El contexto puede incluir documentos oficiales (leyes, reglamentos, instructivos, padrones) y también materiales informativos confiables sobre candidatos, boletas y listas.  
- Si no hay información suficiente en el contexto para responder, decí: “Lo siento, no dispongo de información sobre ese punto.”

**Instrucciones**  
1. **Usá solo la información incluida en la sección de contexto más abajo. No inventes ni completes por fuera.  
2. Siempre que sea posible, **mencioná la fuente** del dato (“Según la Ley X…”, “Según el instructivo…”, “Según el material informativo…”).  
3. Respondé primero de forma **breve y directa**, luego ampliá si es necesario (máximo 2 oraciones).  
4. Usá un lenguaje **neutral**, **claro** y **accesible** para jueces de mesa, fiscales, autoridades y votantes.

**Casos que podés responder**  
- Procedimientos ante situaciones comunes (faltan boletas, DNI no válido, votantes enojados, etc.).  
- Reglas sobre el acceso al local (ropa partidaria, acompañantes, animales, horarios).  
- Información sobre **candidatos, listas y boletas**, si está presente en el contexto.  
- Consultas de **autoridades de mesa**, como:  
  - ¿Qué hacer si no se presenta el presidente?  
  - ¿Quién debe firmar el acta?  
  - ¿Quién reemplaza a quién en caso de ausencia?  
  - ¿Quién puede asistir al escrutinio?  
  - ¿Qué responsabilidad tiene cada cargo en la mesa?  
  - ¿Qué se hace si falta una autoridad o un fiscal?

---

**Contexto**:  
{context}

**Pregunta**:  
{question}

**Respuesta**:
"""

custom_prompt = PromptTemplate.from_template(template)

# Ahora lo pasamos al construir la cadena
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt},
    callbacks=[StdOutCallbackHandler()]  # Este callback muestra en consola lo que se envía y recibe
)

Ahora vamos a levantar esto con Gradio usando la interfaz de Chat -


In [19]:
# Encapsulamos la lógica del chat en una función
def chat(pregunta, historial):
    # Se invoca la cadena de conversación pasando la pregunta del usuario
    resultado = conversation_chain.invoke({"question": pregunta})
    
    # Imprimimos en consola:
    print(resultado["answer"])
    
    # Se devuelve solo la respuesta generada por el modelo
    return resultado["answer"]


In [20]:
# Y en Gradio:
view = gr.ChatInterface(
    chat,
    title="🗳️ ElectoAI – Asistente Electoral 2025",
    description="Consultá sobre normativas, procedimientos y candidaturas en la Ciudad de Buenos Aires.",
    type="messages"
).launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ece5472192c8e27ae6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are **ElectoAI**, an expert electoral assistant for the **2025 legislative elections in the City of Buenos Aires (CABA)**.

**Role & Scope**  
- Proporcionás respuestas **claras**, **precisas** y **neutrales** sobre procedimientos electorales, normativas vigentes, situaciones prácticas en el lugar de votación y candidaturas, basándote únicamente en la información provista en el contexto recuperado.  
- El contexto puede incluir documentos oficiales (leyes, reglamentos, instructivos, padrones) y también materiales informativos confiables sobre candidatos, boletas y listas.  
- Si no hay información suficiente en el contexto para responder, decí: “Lo siento, no dispongo de información sobre ese punto.”

**Instrucciones**  
1. **Usá solo la información incluida en la sección de contexto más abajo. No inventes ni completes por 

# Debugging

Diagnóstico: Chunks más relevantes de una query?

In [25]:
# Primero podemos mirar la numeración de los chunks creados a partir de un archivo. Sabiendo la pregunta, sabriamos cual sería el chunk más relevante y ahí
# vemos si aparece en la búsqueda o que puntuación tiene que hace que no aparezca.
nombre_archivo = "Manual_para_autoridades_mesa_caba_2025.md"

chunks_objetivo = [
    chunk for chunk in chunks
    if nombre_archivo in chunk.metadata["source"]
]

for i, chunk in enumerate(chunks_objetivo):
    # Buscar su índice global dentro de todos los chunks
    index_global = chunks.index(chunk)
    
    print(f"\n--- Chunk {i+1} (índice global: {index_global}) ---\n")
    print(chunk.page_content)



--- Chunk 1 (índice global: 257) ---

---
title: "Manual de Capacitación para Autoridades de Mesa"
subtitle: "Elecciones 2025 - Ciudad Autónoma de Buenos Aires"
---

# Presentación

Este manual tiene como objetivo brindar a las Autoridades de Mesa los conocimientos esenciales para desempeñar su función conforme a la normativa vigente. Se estructura en función de las tareas que deben desarrollarse durante la jornada electoral e incluye información práctica, representaciones gráficas de los materiales y documentos utilizados, y protocolos para gestionar situaciones imprevistas.

# Cuándo votamos

Las elecciones locales en la Ciudad Autónoma de Buenos Aires se realizarán en una fecha distinta a la de las elecciones nacionales. No habrá elecciones Primarias Abiertas, Simultáneas y Obligatorias (PASO). La jornada electoral se llevará a cabo el día domingo del mes de mayo de 2025.

# Qué votamos

--- Chunk 2 (índice global: 258) ---

# Qué votamos

Se elegirá una lista de candidatos/as a Di

In [22]:
query = "Cuándo votamos"

resultados = vectorstore.similarity_search_with_score(query, k=50)

for doc, score in resultados:
    print(f"Score: {score:.2f}")
    print(doc.page_content[:200])
    print("---")


Score: 0.34
#### Artículo 210: Emisión del voto. La emisión del sufragio se lleva a cabo mediante el siguiente procedimiento:
1) Cuando el/la elector/a se encuentre en el puesto de votación o frente al dispositiv
---
Score: 0.34
---
title: "Manual de Capacitación para Autoridades de Mesa"
subtitle: "Elecciones 2025 - Ciudad Autónoma de Buenos Aires"
---

# Presentación

Este manual tiene como objetivo brindar a las Autoridade
---
Score: 0.36
#### Artículo 203: Derecho a votar. Toda persona que figure en el padrón y acredite su identidad mediante la
exhibición del Documento Nacional de Identidad habilitante, en las condiciones establecidas
---
Score: 0.37
#### Artículo 142: Opciones de voto. El dispositivo electrónico del Sistema electrónico de emisión de Boleta
deberá presentar inicialmente al/la elector/a la opción de votar por categoría o por agrupa
---
Score: 0.37
#### Artículo 216: Clausura del acto electoral. A las dieciocho horas (18 hs.) se clausurará el acto electoral y se
disp

In [34]:
# Paso 1: crear el embedding de la query
query = "cuando se vota"
query_embedding = embeddings.embed_query(query)

# Paso 2: elegir un chunk específico
chunk = chunks[257]  # por ejemplo, el chunk que quiero evaluar (ese número es el indice global que encontramos gracias a que más arriba vimos los chunks de cada documento)
chunk_embedding = embeddings.embed_documents([chunk.page_content])[0]

print(chunks[257])
# Opción A: distancia Euclídea (L2)
l2_distance = np.linalg.norm(np.array(query_embedding) - np.array(chunk_embedding))
print(f"Distancia L2: {l2_distance:.4f}")

# Opción B: similitud coseno (más alta = más similar)
cos_sim = np.dot(query_embedding, chunk_embedding) / (
    np.linalg.norm(query_embedding) * np.linalg.norm(chunk_embedding)
)
print(f"Similitud coseno: {cos_sim:.4f}")


page_content='---
title: "Manual de Capacitación para Autoridades de Mesa"
subtitle: "Elecciones 2025 - Ciudad Autónoma de Buenos Aires"
---

# Presentación

Este manual tiene como objetivo brindar a las Autoridades de Mesa los conocimientos esenciales para desempeñar su función conforme a la normativa vigente. Se estructura en función de las tareas que deben desarrollarse durante la jornada electoral e incluye información práctica, representaciones gráficas de los materiales y documentos utilizados, y protocolos para gestionar situaciones imprevistas.

# Cuándo votamos

Las elecciones locales en la Ciudad Autónoma de Buenos Aires se realizarán en una fecha distinta a la de las elecciones nacionales. No habrá elecciones Primarias Abiertas, Simultáneas y Obligatorias (PASO). La jornada electoral se llevará a cabo el día domingo del mes de mayo de 2025.

# Qué votamos' metadata={'source': 'knowledge-base\\Manual autoridades de mesa\\Manual_para_autoridades_mesa_caba_2025.md', 'doc_type':